# Survey Propagation at zero temperature

In [461]:
include("../code/sp.jl")
include("../code/slim_graphs.jl")
using StaticArrays, OffsetArrays, BenchmarkTools

In [402]:
H = readgraph("/tmp/graph9000.txt")
H = readseeds("/tmp/seeds9000.txt", H)
M,N = size(H)
rowperm,colperm = leaf_removal(H);

size(H) = (6600, 9000)
5 removed factors [129, 1515, 1531, 1655, 3471]


In [403]:
x0 = rand((0,1), size(H,2))
s = (-1).^x0
J = 10
init=OffsetArray(MVector{2J+1}(randn(2J+1)), -J:J)
sp1 = survey_propagation(H; field=s, init, y=2.0);

## Updates for variables and factors

In [464]:
b = 21
∂b = nonzeros(sp.X)[nzrange(sp.X, b)]
sp = deepcopy(sp1)
sp2 = deepcopy(sp1)
e1 = update_factor_zeroT_slow!(sp1, b; damp = 0.4)
e2 = update_factor_zeroT!(sp2, b; damp = 0.4)
[sp1.P[∂b][2] sp2.P[∂b][2]]
e1, e2

(0.5924936547636817, 0.5924936547636817)

In [405]:
@btime update_factor_zeroT_slow!($sp1, $b; damp = 0.4)

  1.757 ms (15907 allocations: 499.23 KiB)


4.440892098500626e-16

In [406]:
@btime update_factor_zeroT!($sp1, $b; damp = 0.4)

  17.384 μs (124 allocations: 16.98 KiB)


4.440892098500626e-16

In [467]:
i = 
∂i = nzrange(sp.H, i)
sp = deepcopy(sp1)
e1 = update_var_zeroT_slow!(sp1, i; damp = 0.1, rein = 0.1)
e2 = update_var_zeroT!(sp, i; damp = 0.1, rein = 0.1)
[sp1.survey[i] sp.survey[i]]
[sp1.Q[∂i][2] sp.Q[∂i][2]]
# e1,e2

21×2 Array{Float64,2}:
   0.172396    0.172396
  -1.90871    -1.90871
  -1.58897    -1.58897
  -2.52424    -2.52424
  -0.605793   -0.605793
  -1.1778     -1.1778
  -1.68192    -1.68192
  -4.28022    -4.28022
  -2.9248     -2.9248
  -2.48352    -2.48352
  -3.96883    -3.96883
  -4.24456    -4.24456
  -3.52041    -3.52041
  -4.35056    -4.35056
  -4.39801    -4.39801
  -3.86666    -3.86666
  -2.36743    -2.36743
  -1.88273    -1.88273
  -4.38934    -4.38934
  -3.64829    -3.64829
 -Inf        -Inf

In [455]:
@btime update_var_zeroT_slow!($sp1, $i; damp = 0.4, rein = 1e-3)

  297.501 μs (5349 allocations: 160.33 KiB)


8.881784197001252e-16

In [456]:
@btime update_var_zeroT!($sp1, $i; damp = 0.4, rein = 1e-3)

  5.886 μs (19 allocations: 3.72 KiB)


8.881784197001252e-16

## Run on an instance

## Recursion for the b's

In [356]:
Q = [OffsetArray(randn(2J+1),-J:J) for _ in 1:3];

In [357]:
function Bslow(Q; k=length(Q)-1)
   b = fill(-Inf, -J:J)
    for hs in Iterators.product(fill(-J:J, k)...)
        u = minimum(abs, hs)
        σ = prod(sign, hs)
        for v in 1:u
           b[v*σ] = max(b[v*σ], sum(q[h] for (q,h) in zip(Q,hs))) 
        end
    end
    b
end

function Bfast(Q; k=length(Q)-1)
    b = fill(-Inf, -J:J); b[1:J] .= 0.0
    bnew = fill(-Inf, -J:J); bnew[1:J] .= 0.0
    for n in 1:k
        for u in 1:J
            m1 = maximum(Q[n][u:end])
            m2 = maximum(Q[n][begin:-u])
            bnew[u] = max(m1+b[u], m2+b[-u])
            bnew[-u] = max(m1+b[-u], m2+b[u])
        end
        b .= bnew
    end
    b
end

Bfast (generic function with 1 method)

In [358]:
Bsl = Bslow(Q)
Bfa = Bfast(Q)
Bsl == Bfa

true

## Recursion for the a's

In [388]:
Q = [OffsetArray(rand(2J+1),-J:J) for _ in 1:2];

In [389]:
function Aslow(Q; k=length(Q))
    a = fill(-Inf, -J:J)
    for hs in Iterators.product(fill(-J:J, k)...)
        u = minimum(abs, hs)*prod(sign, hs)
        a[u] = max(a[u], sum(q[h] for (q,h) in zip(Q,hs))) 
    end
    a
end

function Pifast(Q)
    a = Aslow(Q, k=length(Q)-1)
    b = Bslow(Q, k=length(Q)-1)
    p = fill(-Inf, -J:J)
    q = Q[end]
    for u in 1:J-1
        m1 = maximum(q[u+1:end])
        m2 = maximum(q[begin:-u-1])
        p[u] = max(q[u]+b[u], q[-u]+b[-u], m1+a[u], m2+a[-u])
        p[-u] = max(q[u]+b[-u], q[-u]+b[u], m1+a[-u], m2+a[u])
    end
    p[J] = max(q[J]+b[J], q[-J]+b[-J])
    p[-J] = max(q[J]+b[-J], q[-J]+b[J])
    # u=0
    sumqstar = 0.0
    qmax = -Inf
    for j in 1:length(Q)
        qstar = maximum(Q[j])
        sumqstar += qstar
        qmax = max(qmax, Q[j][0] - qstar)
    end
    p[0] = sumqstar + qmax
    p
end
    
function Afast(Q, k=length(Q))
    p = fill(-Inf, -J:J)
    b = fill(-Inf, -J:J); b[1:J] .= 0.0
    pnew = fill(-Inf, -J:J)
    bnew = fill(-Inf, -J:J); bnew[1:J] .= 0.0
    sumqstar = 0.0
    qmax = -Inf
    for j in 1:k
        q = Q[j]
        # update for u=0
        qstar = maximum(q)
        sumqstar += qstar
        qmax = max(qmax, q[0] - qstar)
        # update p and b
        for u in 1:J-1
            m1 = maximum(q[u+1:end])
            m2 = maximum(q[begin:-u-1])
            pnew[u] = max(q[u]+b[u], q[-u]+b[-u], m1+p[u], m2+p[-u])
            pnew[-u] = max(q[u]+b[-u], q[-u]+b[u], m1+p[-u], m2+p[u])
            n1 = max(m1, q[u])
            n2 = max(m2, q[-u])
            bnew[u] = max(n1+b[u], n2+b[-u])
            bnew[-u] = max(n1+b[-u], n2+b[u])
        end
        pnew[J] = bnew[J] = max(q[J]+b[J], q[-J]+b[-J])
        pnew[-J] = bnew[-J] = max(q[J]+b[-J], q[-J]+b[J])
        b .= bnew
        p .= pnew
    end
    p[0] = sumqstar + qmax
    p        
end
    
function Afast_old(Q, k=length(Q))
    a = fill(-Inf, -J:J)
    b = fill(-Inf, -J:J); b[1:J] .= 0.0
    anew = fill(-Inf, -J:J)
    bnew = fill(-Inf, -J:J); bnew[1:J] .= 0.0
    for n in 1:k
        q = Q[n]
        # update a
        for u in 1:J-1
            m1 = maximum(q[u+1:end])
            m2 = maximum(q[begin:-u-1])
            anew[u] = max(q[u]+b[u], q[-u]+b[-u], m1+a[u], m2+a[-u])
            anew[-u] = max(q[u]+b[-u], q[-u]+b[u], m1+a[-u], m2+a[u])
        end
        anew[J] = max(q[J]+b[J], q[-J]+b[-J])
        anew[-J] = max(q[J]+b[-J], q[-J]+b[J])
        # update b
        for u in 1:J
            m1 = maximum(q[u:end])
            m2 = maximum(q[begin:-u])
            bnew[u] = max(m1+b[u], m2+b[-u])
            bnew[-u] = max(m1+b[-u], m2+b[u])
        end
        b .= bnew
        a .= anew
    end
    a        
end

Afast_old (generic function with 2 methods)

In [390]:
Asl = Aslow(Q, k=length(Q))
Pfast = Pifast(Q)
Afa = Afast(Q)
[Asl Pfast Afa]

21×3 Array{Float64,2}:
 0.999569  0.999569  0.999569
 1.18077   1.18077   1.18077
 1.50147   1.50147   1.50147
 1.76321   1.76321   1.76321
 1.5637    1.5637    1.5637
 1.70259   1.70259   1.70259
 1.78917   1.78917   1.78917
 1.83508   1.83508   1.83508
 1.63741   1.63741   1.63741
 1.73482   1.73482   1.73482
 1.67677   1.67677   1.67677
 1.77491   1.77491   1.77491
 1.61045   1.61045   1.61045
 1.80812   1.80812   1.80812
 1.77844   1.77844   1.77844
 1.79017   1.79017   1.79017
 1.47612   1.47612   1.47612
 1.67563   1.67563   1.67563
 1.59641   1.59641   1.59641
 1.14453   1.14453   1.14453
 1.07134   1.07134   1.07134

## Max-Sum convolution for variable update

In [231]:
function msc_slow(P, J=10, s=1)
    Pnew = copy(P[2:end])
    f = min(-J, sum(firstindex(p) for p in Pnew)+s)
    l = max(J,sum(lastindex(p) for p in Pnew)+s)
    q = fill(Inf, f:l)
    for us in Iterators.product(fill(-J:J, length(Pnew))...)
        h = sum(us) + s
        q[h] = min(q[h],abs(h) - sum(abs.(u)+p[u] for (p,u) in zip(Pnew,us)))
    end
    replace!(q, Inf => -Inf)
    # clamp
    q[J] = maximum(q[J:end])
    q[-J] = maximum(q[begin:-J])
    q[-J:J]
end

function msc_fast(P, J=10, s=1)
    Pnew = [abs.(OffsetArray(-J:J, -J:J)) + p for p in P[2:end]]
    f = fill(0.0, s:s)
    for n in eachindex(Pnew)
        f = msc(f, Pnew[n])
    end  
    q = fill(-Inf, -J:J)
    for h in eachindex(f)
        q[clamp(h,-J,J)] = max(q[clamp(h,-J,J)], abs(h) - f[h])
    end
        @show q_fast=q
    q  
end
    
function msc_faster(P, J=10, s=1)
    Pnew = [abs.(OffsetArray(-J:J, -J:J)) + p for p in P[2:end]]
    init = fill(0.0, s:s)
    Q = [fill(0.0, 0:0) for a ∈ 1:length(Pnew)]
    f = cavity!(Q, Pnew, msc, init)
    q = fill(-Inf, -J:J)
    for h in eachindex(f)
        q[clamp(h,-J,J)] = max(q[clamp(h,-J,J)], abs(h) - f[h])
    end
    @show q_faster = q
    q
end

msc_faster (generic function with 3 methods)